In [2]:
# 🚀 Gaussian Noise Image Generator for U-Net Denoising Project
import os
import cv2
import numpy as np
from google.colab import drive

# 👇 Mount Google Drive (optional, if dataset is in Drive)
drive.mount('/content/drive')

# 📂 Define paths (change these if needed)
clean_path = '/content/drive/MyDrive/U-Net-Denoising-Project/dataset/clean'
noisy_path = '/content/drive/MyDrive/U-Net-Denoising-Project/dataset/noisy'

# ✅ Create noisy folder if it doesn't exist
os.makedirs(noisy_path, exist_ok=True)

# 🔧 Function to add Gaussian noise safely
def add_gaussian_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    noisy = image.astype(np.float32) + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

# 🔁 Process each clean image
image_count = 0
for filename in os.listdir(clean_path):
    file_path = os.path.join(clean_path, filename)
    img = cv2.imread(file_path)
    if img is None:
        print(f"❌ Skipped: {filename}")
        continue
    img = cv2.resize(img, (128, 128))
    noisy_img = add_gaussian_noise(img)
    save_path = os.path.join(noisy_path, filename)
    cv2.imwrite(save_path, noisy_img)
    image_count += 1

print(f"✅ Successfully generated {image_count} noisy images in: {noisy_path}")


Mounted at /content/drive
✅ Successfully generated 272 noisy images in: /content/drive/MyDrive/U-Net-Denoising-Project/dataset/noisy


In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam

# 📁 Define dataset paths
clean_path = '/content/drive/MyDrive/U-Net-Denoising-Project/dataset/clean'
noisy_path = '/content/drive/MyDrive/U-Net-Denoising-Project/dataset/noisy'

# 📦 Load dataset
def load_dataset(clean_path, noisy_path, img_size=(128, 128)):
    clean_imgs = []
    noisy_imgs = []

    for fname in os.listdir(clean_path):
        clean_img = load_img(os.path.join(clean_path, fname), target_size=img_size)
        noisy_img = load_img(os.path.join(noisy_path, fname), target_size=img_size)

        clean_imgs.append(img_to_array(clean_img) / 255.0)
        noisy_imgs.append(img_to_array(noisy_img) / 255.0)

    return np.array(noisy_imgs), np.array(clean_imgs)

# 🧠 Build U-Net model
def build_unet(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D(2)(c1)

    c2 = Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D(2)(c2)

    c3 = Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = MaxPooling2D(2)(c3)

    c4 = Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = Conv2D(512, 3, activation='relu', padding='same')(c4)
    p4 = MaxPooling2D(2)(c4)

    # Bottleneck
    c5 = Conv2D(1024, 3, activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, 3, activation='relu', padding='same')(c5)

    # Decoder
    u6 = Conv2DTranspose(512, 2, strides=2, padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, 3, activation='relu', padding='same')(u6)
    c6 = Conv2D(512, 3, activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, 2, strides=2, padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, 3, activation='relu', padding='same')(u7)
    c7 = Conv2D(256, 3, activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, 2, strides=2, padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, 3, activation='relu', padding='same')(u8)
    c8 = Conv2D(128, 3, activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, 2, strides=2, padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, 3, activation='relu', padding='same')(u9)
    c9 = Conv2D(64, 3, activation='relu', padding='same')(c9)

    outputs = Conv2D(3, 1, activation='sigmoid')(c9)

    model = Model(inputs, outputs)
    return model

# 🚀 Load data
X_noisy, Y_clean = load_dataset(clean_path, noisy_path)
print(f"Loaded {len(X_noisy)} image pairs.")

# 🏗️ Build model
model = build_unet()
model.compile(optimizer=Adam(), loss='mean_squared_error')
model.summary()

# 🧠 Train model
model.fit(X_noisy, Y_clean, batch_size=8, epochs=20, validation_split=0.1)

# 💾 Save model to Drive
model_path = '/content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5'
model.save(model_path)
print(f"✅ Model saved to {model_path}")


Loaded 272 image pairs.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 8, 8, 512) │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 8, 8,      │  4,719,616 │ max_pooling2d_3[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 8, 8,      │  9,438,208 │ conv2d_8[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 16, 16,    │  2,097,664 │ conv2d_9[0][0]    │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 16,    │          0 │ conv2d_transpose

 Total params: 31,031,875 (118.38 MB)

 Trainable params: 31,031,875 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 58s 995ms/step - loss: 0.0720 - val_loss: 0.0142
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 144ms/step - loss: 0.0115 - val_loss: 0.0070
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 149ms/step - loss: 0.0067 - val_loss: 0.0060
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 150ms/step - loss: 0.0066 - val_loss: 0.0052
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 150ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 147ms/step - loss: 0.0055 - val_loss: 0.0049
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 147ms/step - loss: 0.0050 - val_loss: 0.0046
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 147ms/step - loss: 0.0047 - val_loss: 0.0049
Epoch 9/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - loss: 0.0044 - val_loss: 0.0046
Epoch 10/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 153ms/step - loss: 0.0046 - val_loss: 0.0047
Epoch 11/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 150ms/step - loss: 0.0043 - val_loss: 0.0042
Epoch 12/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 150ms/ste

✅ Model saved to /content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5


In [6]:
model_path = '/content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5'
print(model_path)


/content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5


In [7]:
import os

if os.path.exists('/content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5'):
    print("✅ Model exists.")
else:
    print("❌ Model not found.")


✅ Model exists.


In [8]:
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5'
model = load_model(model_path)
print("✅ Model loaded successfully.")


✅ Model loaded successfully.


In [ ]:
!pip install streamlit
!pip install localtunnel

In [12]:
%%writefile streamlit_app.py
import streamlit as st
import numpy as np
import cv2
from PIL import Image
from tensorflow.keras.models import load_model

# Load model
model_path = "/content/drive/MyDrive/U-Net-Denoising-Project/model/unet_model.h5"
model = load_model(model_path)

# Function to preprocess image
def preprocess_image(uploaded_file):
    img = Image.open(uploaded_file).convert("RGB")
    img = img.resize((128, 128))
    img_array = np.array(img) / 255.0
    return img_array, img

# Function to denoise
def denoise_image(image_array):
    input_img = np.expand_dims(image_array, axis=0)  # Add batch dimension
    denoised_img = model.predict(input_img)[0]
    denoised_img = np.clip(denoised_img * 255, 0, 255).astype(np.uint8)
    return denoised_img

# Streamlit UI
st.title("🧼 Image Denoising using U-Net")

uploaded_file = st.file_uploader("Upload a noisy image", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    st.image(uploaded_file, caption="Noisy Input", use_column_width=True)
    img_array, orig_img = preprocess_image(uploaded_file)
    denoised = denoise_image(img_array)
    st.image(denoised, caption="Denoised Output", use_column_width=True)


Overwriting streamlit_app.py


In [13]:
!pip install streamlit
!pip install pyngrok

from pyngrok import ngrok

# Run Streamlit app
public_url = ngrok.connect(port=8501)
print(f"🌐 Streamlit App Link: {public_url}")

!streamlit run streamlit_app.py --server.port 8501 > /dev/null 2>&1


ERROR:pyngrok.process.ngrok:t=2025-07-12T15:58:00+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-12T15:58:00+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-12T15:58:00+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [14]:
!ngrok config add-authtoken 2zmV2SkoUB3JUZhUFxEfYxBCJC0_3DssY2Coe8qJpeh6MjCZa


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from pyngrok import ngrok

# Start Streamlit tunnel manually for port 8501
public_url = ngrok.connect(addr="http://localhost:8501", proto="http")
print(f"🌐 Streamlit App Link: {public_url}")



🌐 Streamlit App Link: NgrokTunnel: "https://6179c0b48a22.ngrok-free.app" -> "http://localhost:8501"


In [1]:
from pyngrok import ngrok
public_url = ngrok.connect(addr="http://localhost:8501", proto="http")
print(f"🌐 Streamlit App Link: {public_url}")


🌐 Streamlit App Link: NgrokTunnel: "https://0ccfc7c986e5.ngrok-free.app" -> "http://localhost:8501"


In [2]:
!streamlit run streamlit_app.py --server.port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.150.143:8501



  Stopping...
^C


In [3]:
from pyngrok import ngrok

public_url = ngrok.connect(addr="http://localhost:8501", proto="http")
print(f"🌐 Streamlit App Link: {public_url}")


🌐 Streamlit App Link: NgrokTunnel: "https://2d3dbcfbad80.ngrok-free.app" -> "http://localhost:8501"


In [6]:
!nohup streamlit run streamlit_app.py --server.port 8501 > /dev/null 2>&1 &


In [7]:
from pyngrok import ngrok

public_url = ngrok.connect(addr="http://localhost:8501", proto="http")
print(f"🌐 Streamlit App Link: {public_url}")


🌐 Streamlit App Link: NgrokTunnel: "https://037a4cb2f252.ngrok-free.app" -> "http://localhost:8501"
